In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../')
from seqmodel.bunch import Bunch

In [5]:
import time
import copy
import tensorflow as tf
import numpy as np
from seqmodel.model import rnn_module
from seqmodel.model import graph_util

In [21]:
tf.reset_default_graph()
batch_size = 4
max_time = 5
dim = 3
input_data = np.random.randn(max_time, batch_size, dim).astype(np.float32)
seq_len_data = np.array([4, 3, 2, 1], dtype=np.int32)
inputs = tf.constant(input_data, dtype=tf.float32)
# inputs = tf.slice(inputs, [max_time, 0, 0], [-1, -1, -1])
seq_len = tf.constant(seq_len_data)
rnn_template = tf.make_template('share_rnn', tf.nn.dynamic_rnn, create_scope_now_=True)
# create share_rnn
share_rnn_opt = rnn_module.BasicRNNModule.default_opt()
share_rnn_opt.rnn_cell.cell_opt.num_units = 2
share_rnn_opt.logit=None
enc_opt = copy.deepcopy(share_rnn_opt)
enc = rnn_module.BasicRNNModule(enc_opt, name='encoder')(inputs, seq_len, rnn_fn=rnn_template)
dec_opt = copy.deepcopy(share_rnn_opt)
dec_opt.logit = graph_util.default_logit_opt()
dec = rnn_module.BasicRNNModule(dec_opt, name='decoder')(
    inputs, seq_len, rnn_fn=rnn_template, initial_state=enc.final_state)
for v in tf.trainable_variables():
    print(v.name)

share_rnn/rnn/basic_lstm_cell/weights:0
share_rnn/rnn/basic_lstm_cell/biases:0
decoder/logit_w:0
decoder/logit_b:0


In [22]:
sess_config = tf.ConfigProto(log_device_placement=False,
                             device_count={'GPU': 0})
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    fetch = Bunch(inputs=inputs,
                  enc=Bunch(state=enc.final_state, outputs=enc.cell_output),
                  dec=Bunch(state=dec.final_state, outputs=dec.cell_output))
    result = sess.run(fetch)

In [23]:
result.enc

Bunch(outputs=array([[[-0.05551952, -0.17745341],
        [ 0.45945889,  0.20322596],
        [ 0.2903592 ,  0.20640534],
        [-0.04771867, -0.2949748 ]],

       [[-0.06333468, -0.08701437],
        [ 0.29879823,  0.11412661],
        [ 0.29974586,  0.04292478],
        [ 0.        ,  0.        ]],

       [[-0.05984084, -0.11516758],
        [ 0.35779566,  0.14393196],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.06619542, -0.00258966],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]], dtype=float32), state=LSTMStateTuple(c=array([[ 0.11464296, -0.00623642],
       [ 0.75493264,  0.29318127],
       [ 0.67389071,  0.14293405],
       [-0.17841384, -0.44135126]], dtype=float32), h=array([[ 0.06619542, -0.00258966],
       [ 0.35779566,  0

In [24]:
result.dec

Bunch(outputs=array([[[-0.01655734, -0.18060914],
        [ 0.58960813,  0.34188023],
        [ 0.46440688,  0.27818632],
        [-0.08611448, -0.50745291]],

       [[-0.02240832, -0.08838212],
        [ 0.351872  ,  0.17394544],
        [ 0.35927469,  0.05469111],
        [ 0.        ,  0.        ]],

       [[-0.01777641, -0.11658371],
        [ 0.39684907,  0.19325076],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.09870893, -0.00361167],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]], dtype=float32), state=LSTMStateTuple(c=array([[ 0.17343497, -0.00864151],
       [ 0.8960253 ,  0.40930146],
       [ 0.93489581,  0.1836054 ],
       [-0.33050391, -0.82711327]], dtype=float32), h=array([[ 0.09870893, -0.00361167],
       [ 0.39684907,  0

In [25]:
result.inputs

array([[[-1.76871634,  0.97102064, -1.44185841],
        [ 0.38082048,  1.58393133,  0.88840854],
        [-0.07286013,  1.02017522,  1.12544453],
        [-2.11796188,  0.16951987, -0.77417612]],

       [[ 0.48452482, -0.59376633, -0.54090321],
        [ 0.63103467,  0.27867132, -0.49724039],
        [ 1.76447403,  0.17218235, -1.51190579],
        [-0.40062335, -0.35411108,  0.87544507]],

       [[-0.13708884,  0.40645659, -0.81410635],
        [ 0.49890959,  0.45203853, -0.10353998],
        [ 1.00506556,  0.29860762,  1.09288502],
        [-0.38584483,  2.34337783, -0.0965653 ]],

       [[ 0.63455939, -0.37998897,  0.69583702],
        [ 0.63661444,  0.64823347,  0.1341929 ],
        [ 0.49124098, -1.10873079,  0.11810127],
        [-0.58233732,  0.27009308, -0.41079408]],

       [[-0.76649064, -1.01023149, -0.17885061],
        [-1.41704857,  0.8663041 ,  0.8057878 ],
        [ 1.54822505, -0.07733702,  0.98164129],
        [ 0.63696241,  1.68602681, -0.68549204]]], dtype=floa